In [ ]:
dbutils.widgets.text('src_catalog', '', '01 Source Catalog')
dbutils.widgets.text('src_schema', '', '02 Source Schema')
dbutils.widgets.text('src_table', '', '03 Source Table')
dbutils.widgets.text('partition_col', '', '04 Source Partition Column')
dbutils.widgets.text('tgt_catalog', '', '05 Target Catalog')
dbutils.widgets.text('tgt_schema', '', '06 Target Schema')
dbutils.widgets.text('tgt_table', '', '07 Target Table')

In [ ]:
from lakefed_ingest.main import *

In [ ]:
src_catalog = dbutils.widgets.get('src_catalog')
src_schema = dbutils.widgets.get('src_schema')
src_table = dbutils.widgets.get('src_table')
partition_col = dbutils.widgets.get('partition_col')
tgt_catalog = dbutils.widgets.get('tgt_catalog')
tgt_schema = dbutils.widgets.get('tgt_schema')
tgt_table = dbutils.widgets.get('tgt_table')

In [ ]:
partition_spec = get_partition_spec_sqlserver(
    catalog=src_catalog,
    schema=src_schema,
    table=src_table,
    partition_col=partition_col,
    partition_size_mb=200,
)

print(partition_spec)

In [ ]:
partition_list = get_partition_list(
    partition_col,
    **partition_spec
)

# Write partition list to table
spark.sql(f'use catalog {tgt_catalog}')
spark.sql(f'use schema {tgt_schema}')
partition_list_to_table(partition_list, f'{tgt_table}_partition_list', partition_spec['num_partitions'])

In [ ]:
batch_list_qry = f"""\
select array_agg(distinct batch_id) as batch_list
from {tgt_table}_partition_list
"""

batch_list = spark.sql(batch_list_qry).collect()[0][0]
batch_list.sort()
print(f'batch_list: {batch_list}')

# Assign batch list to job task value. The list can be iterated in a subsequent task.
dbutils.jobs.taskValues.set(key="batch_list", value=batch_list)